# How to map query results to objects

To map query results to objects with C# and Dapper, you can use the <code>Query</code> or <code>QueryAsync</code> methods of the Dapper extension. These methods allow you to execute SQL queries against the database and map the results to objects using reflection or other techniques.

Mapping query results to objects with C# and Dapper is a process of transforming the data returned by a SQL query into C# objects that can be manipulated in code. Dapper is a micro-ORM (object-relational mapper) that simplifies this process by using reflection and dynamic methods to map the columns of the query result to the properties of the object.

One of the features of Dapper is that it does not require the data columns and the object properties to have the same name or order. Dapper can automatically match the columns to the properties based on their names, ignoring case and underscores. For example, a column named customer_id can be mapped to a property named CustomerId without any additional configuration. Dapper can also handle complex mappings, such as one-to-one, one-to-many, and many-to-many relationships, by using the <code>splitOn</code> parameter and the <code>QueryMultiple</code> method.

However, there are some cases where the automatic mapping may not work as expected, such as when the column names are ambiguous, when the object has multiple constructors, or when the object has nested properties. In these cases, Dapper provides some options to customize the mapping, such as using aliases in the SQL query, specifying the constructor to use, or using the <code>Dapper.Contrib</code> extension to define custom mappings. These options can help to ensure that the query results are mapped to the objects correctly and efficiently.

For example, if you want to map the results of the following SQL query to a Product class:


| Object | Query Result |  
|---|---|
| CountryRegionCode | CountryRegionCode |
| CountryRegionName | CountryRegionName | 
| Comment |   |
|    | Test |  | 


<code>CountryRegionCode</code> and <code>CountryRegionName</code> are perfectly mapped; while Comment and Test are unmatched therefore will be ignored.

The following example demonstrates the concept of query mapping.


In [1]:
// Connection string
#load "AppSettings.cs"

In [2]:
#r "nuget:Microsoft.Data.SqlClient"
#r "nuget:Dapper"
using Microsoft.Data.SqlClient;
using Dapper;

// Define a class for the CountryRegion table
public class CountryRegion
{         
    public string CountryRegionCode { get; set; }
    public string CountryRegionName { get; set; }  
    public string Comment { get; set; }  
}

// Create a SqlConnection object using the connection string
using (SqlConnection connection = new SqlConnection(connectionString))
{
    // Open the connection
    connection.Open();

    // Define a SQL query to select some columns from the Person.CountryRegion table
    string sql = "SELECT CountryRegionCode, Name as CountryRegionName, 'Ignore me' AS Test FROM Person.CountryRegion";

    // Execute the query and map the results to a list of CountryRegion objects
    var countryRegions = connection.Query<CountryRegion>(sql);
    foreach (CountryRegion countryRegion in countryRegions)
    {
        Console.WriteLine($"{countryRegion.CountryRegionCode} - {countryRegion.CountryRegionName} - {countryRegion.Comment}");
    }
}

Installed Packages Dapper, 2.1.35 Microsoft.Data.SqlClient, 5.2.0

AF - Afghanistan - 
AL - Albania - 
DZ - Algeria - 
AS - American Samoa - 
AD - Andorra - 
AO - Angola - 
AI - Anguilla - 
AQ - Antarctica - 
AG - Antigua and Barbuda - 
AR - Argentina - 
AM - Armenia - 
AW - Aruba - 
AU - Australia - 
AT - Austria - 
AZ - Azerbaijan - 
BS - Bahamas, The - 
BH - Bahrain - 
BD - Bangladesh - 
BB - Barbados - 
BY - Belarus - 
BE - Belgium - 
BZ - Belize - 
BJ - Benin - 
BM - Bermuda - 
BT - Bhutan - 
BO - Bolivia - 
BA - Bosnia and Herzegovina - 
BW - Botswana - 
BV - Bouvet Island - 
BR - Brazil - 
IO - British Indian Ocean Territory - 
BN - Brunei - 
BG - Bulgaria - 
BF - Burkina Faso - 
BI - Burundi - 
KH - Cambodia - 
CM - Cameroon - 
CA - Canada - 
CV - Cape Verde - 
KY - Cayman Islands - 
CF - Central African Republic - 
TD - Chad - 
CL - Chile - 
CN - China - 
CX - Christmas Island - 
CC - Cocos (Keeling) Islands - 
CO - Colombia - 
KM - Comoros - 
CG - Congo - 
CD - Congo (DRC) - 
CK - Cook Islands - 
CR - Costa Rica - 
CI - Côte d'Ivoire - 
HR -

Note that CountryRegionCode and CountryRegionName are mapped, while Comment and Test are not.

To use map and <code>splitOn</code> with Dapper, you need to understand the concept of multi-mapping. Multi-mapping is a feature of Dapper that allows you to map a single row of data to multiple objects. For example, if you have a query that joins two tables, such as Person.StateProvince and Person.CountryRegion, you can use multi-mapping to map each row to a StateProvince object and a CountryRegion object.

To use multi-mapping, you need to use the <code>Query</code> method with multiple type parameters, such as <code>Query<StateProvince, CountryRegion, (StateProvince, CountryRegion)></code>. The first two type parameters are the types of the objects you want to map to, and the last one is the type of the result you want to return. You also need to provide a mapping function that tells Dapper how to combine the two objects into one result. For example, you can use a tuple to store the StateProvince and CountryRegion objects, or you can use a custom class that has properties for both.

Here is an example of how to use map and <code>splitOn</code> with Dapper using Person.StateProvince and Person.CountryRegion tables:

In [3]:
// Define a class for the StateProvince table
public class StateProvince
{
    public int StateProvinceID { get; set; }
    public string StateProvinceCode { get; set; }
    public string StateProvinceName { get; set; }   
    public CountryRegion CountryRegion { get; set; }    
}

List<(StateProvince, CountryRegion)> stateProvinces = new List<(StateProvince, CountryRegion)>();
using (var connection = new SqlConnection(connectionString))
{
    var query = @"select sp.StateProvinceID, sp.StateProvinceCode, sp.Name as StateProvinceName, sp.CountryRegionCode, cr.Name as CountryRegionName
                  from Person.StateProvince sp
                  join Person.CountryRegion cr on sp.CountryRegionCode = cr.CountryRegionCode";
    stateProvinces = connection.Query<StateProvince, CountryRegion, (StateProvince, CountryRegion)>(query, 
                       (sp, cr) => (sp, cr), splitOn: "CountryRegionCode").ToList();   
}

// Display the orders in the list
foreach ((StateProvince, CountryRegion) order in stateProvinces)
{
    Console.WriteLine($"{order.Item1.StateProvinceCode}: {order.Item1.StateProvinceName} - {order.Item2.CountryRegionCode} : {order.Item2.CountryRegionName}");
}


AB : Alberta - CA : Canada
AK : Alaska - US : United States
AL : Alabama - US : United States
AR : Arkansas - US : United States
AS : American Samoa - AS : American Samoa
AZ : Arizona - US : United States
BC : British Columbia - CA : Canada
BY : Bayern - DE : Germany
CA : California - US : United States
CO : Colorado - US : United States
CT : Connecticut - US : United States
DC : District of Columbia - US : United States
DE : Delaware - US : United States
ENG: England - GB : United Kingdom
FL : Florida - US : United States
FM : Micronesia - FM : Micronesia
GA : Georgia - US : United States
GU : Guam - US : United States
HE : Hessen - DE : Germany
HH : Hamburg - DE : Germany
HI : Hawaii - US : United States
IA : Iowa - US : United States
ID : Idaho - US : United States
IL : Illinois - US : United States
IN : Indiana - US : United States
KS : Kansas - US : United States
KY : Kentucky - US : United States
LA : Louisiana - US : United States
LB : Labrador - CA : Canada
MA : Massachusetts -

The <code>splitOn</code> parameter is used to tell Dapper where to split the data into different objects. By default, Dapper assumes that the split point is the Id column, which means that anything before the Id column will be mapped to the first object, and anything after the Id column will be mapped to the second object. However, you can specify a different column name or a list of column names to use as split points. For example, if you have a query that joins three tables, such as Person.StateProvince, Person.CountryRegion, and Person.Address, you can use <code>splitOn</code>: “CountryRegionID, AddressID” to tell Dapper to split the data on the CountryRegionID and AddressID columns.